 <a class="anchor" id="About_Data"></a>
# <p style="padding:10px;background-color:#2e5551;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">2.Préprocessing
</p> 

In [1]:
from tqdm import tqdm

# Bibliothèques pour le traitement de l'arabe
import pyarabic.araby as araby
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.normalize import normalize_unicode, normalize_alef_maksura_ar
from camel_tools.utils.dediac import dediac_ar

# Pour visualiser les résultats


import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import string
import nltk

nltk.download('punkt', download_dir=r'C:\Users\hp\AppData\Roaming\nltk_data')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
from textblob import Word  # ✅ Import the Word class
from sklearn import preprocessing


import sys








[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
# Configuration pour l'affichage en arabe
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.1  Chargement du dataset original.</p>

In [3]:
df= pd.read_csv(r"C:\Users\hp\Desktop\pfemaster\dataset_cleaned.csv", sep=',', encoding='utf-8')

In [4]:
df


,text,Dialect
0,عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه,MA
1,الله يخلف عليك سيدي محمد,MA
2,هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...,MA
3,امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم,MA
4,اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...,MA
...,...,...
24697,الله يدمر هم,TN
24698,يا سي امير راك ماسط ماتضحكش جملا والله,TN
24699,الب منظرك وهف ليييييك يكب سعدك علي ماهو مكبوب...,TN
24700,هذي انسانه فاشله نجحت بالفساد لتعمل فيه ربي يهدي,TN


<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.2 supression demoji .</p>

In [5]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [6]:
df['text'] = df['text'].apply(remove_emoji)

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.3 remove_repeating_characters .</p>

In [7]:
def remove_repeating_characters(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub(r'(.)\1+', r'\1', text)
    return text

In [8]:
df['text'].apply(remove_repeating_characters)

0             عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
1                                  اله يخلف عليك سيدي محمد
2        هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...
3          امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
4        اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...
                               ...                        
24697                                          اله يدمر هم
24698                يا سي امير راك ماسط ماتضحكش جملا واله
24699    الب منظرك وهف ليك يكب سعدك علي ماهو مكبوب تو ا...
24700     هذي انسانه فاشله نجحت بالفساد لتعمل فيه ربي يهدي
24701                              جعفور عاطي لاناقه كارها
Name: text, Length: 24702, dtype: object

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.4 remove_punctuations.</p>

In [9]:

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [10]:
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [11]:
df['text'] = df['text'].apply(remove_punctuations)

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.5 remove_non_arabic .</p>

In [12]:

def remove_non_arabic(text):
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = re.sub('([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+',' ',text)
    text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)
    return text

In [13]:
df['text'].apply(remove_non_arabic)

0             عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
1                                 الله يخلف عليك سيدي محمد
2        هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...
3          امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
4        اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...
                               ...                        
24697                                         الله يدمر هم
24698             يا سي امير راك ماسط ماتضحكش جملا والله  
24699    الب منظرك وهف ليييييك  يكب سعدك علي ماهو مكبوب...
24700     هذي انسانه فاشله نجحت بالفساد لتعمل فيه ربي يهدي
24701                             جعفور عاطي للاناقه كارها
Name: text, Length: 24702, dtype: object

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.6 remove_diacritics .</p>

In [14]:
def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', text)
    text = araby.strip_diacritics(text)
    text = araby.strip_shadda(text)
    text = araby.strip_tashkeel(text)
    return text

In [15]:
df['text'].apply(remove_diacritics)

0             عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
1                                 الله يخلف عليك سيدي محمد
2        هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...
3          امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
4        اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...
                               ...                        
24697                                         الله يدمر هم
24698             يا سي امير راك ماسط ماتضحكش جملا والله  
24699    الب منظرك وهف ليييييك  يكب سعدك علي ماهو مكبوب...
24700     هذي انسانه فاشله نجحت بالفساد لتعمل فيه ربي يهدي
24701                             جعفور عاطي للاناقه كارها
Name: text, Length: 24702, dtype: object

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.7 Fonction de nettoyage complète .</p>

In [16]:

def clean_tweet(text):
    if pd.isna(text):
        return ""
    
   
    # Supprimer les URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Supprimer les mentions (@utilisateur)
    text = re.sub(r'@\w+', '', text)
    
    # Supprimer les hashtags (#mot) - on peut conserver le mot sans le #
    text = re.sub(r'#(\w+)', r'\1', text)
    
    
    
    # Normaliser les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

In [17]:
df['text_1'] = df['text'].apply(clean_tweet)

In [18]:
# Afficher quelques exemples avant/après nettoyage
print("Exemples de tweets avant et après nettoyage:")
for i in range(min(10, len(df))):
    print(f"Original: {df['text'].iloc[i]}")
    print(f"Nettoyé: {df['text_1'].iloc[i]}")
    print("-" * 50)

Exemples de tweets avant et après nettoyage:
Original: عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
Nettoyé: عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
--------------------------------------------------
Original: الله يخلف عليك سيدي محمد
Nettoyé: الله يخلف عليك سيدي محمد
--------------------------------------------------
Original: هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودليل انعدام غرام ديال المنطق
Nettoyé: هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودليل انعدام غرام ديال المنطق
--------------------------------------------------
Original: امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
Nettoyé: امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
--------------------------------------------------
Original: اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما بقات هضره عليها
Nettoyé: اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما بقات هضره عليها
--------------------------------------------------
Original: انفلوانزا الخنازير مسؤول عليها وزير الصحه والحكومه والدوله ياك والحمي القلاعيه واش نقدرو 

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.8 normalisation.</p>

In [19]:
def normalize_text(text):
    if pd.isna(text) or text == "":
        return ""
    

    # Supprimer les répétitions de lettres (plus de 2 occurrences consécutives)
    # Par exemple: سسسسسلام -> سسلام
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    return text

In [20]:
df['tweets_normalized'] = df['text_1'].apply(normalize_text)

In [21]:
print("Exemples de tweets après normalisation:")
for i in range(min(12, len(df))):
    print(f"Nettoyé: {df['text_1'].iloc[i]}")
    print(f"Normalisé: {df['tweets_normalized'].iloc[i]}")
    print("-" * 50)

Exemples de tweets après normalisation:
Nettoyé: عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
Normalisé: عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
--------------------------------------------------
Nettoyé: الله يخلف عليك سيدي محمد
Normalisé: الله يخلف عليك سيدي محمد
--------------------------------------------------
Nettoyé: هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودليل انعدام غرام ديال المنطق
Normalisé: هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودليل انعدام غرام ديال المنطق
--------------------------------------------------
Nettoyé: امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
Normalisé: امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
--------------------------------------------------
Nettoyé: اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما بقات هضره عليها
Normalisé: اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما بقات هضره عليها
--------------------------------------------------
Nettoyé: انفلوانزا الخنازير مسؤول عليها وزير الصحه والحكومه والدوله ياك والحمي القلاعيه واش نقدرو ن

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.9 tokenisation.</p>

In [22]:
def tokenize_arabic(text):
    if pd.isna(text) or text == "":
        return []
    
    # Tokenisation simple par espace pour l'arabe/darija
    # On peut aussi utiliser NLTK avec des règles personnalisées
    tokens = word_tokenize(text)
    
    return tokens

In [23]:
df['tokens'] = df['tweets_normalized'].apply(tokenize_arabic)


In [24]:
print("Exemples de tweets tokenisés:")
for i in range(min(5, len(df))):
    print(f"Normalisé: {df['tweets_normalized'].iloc[i]}")
    print(f"Tokens: {df['tokens'].iloc[i]}")
    print("-" * 50)

Exemples de tweets tokenisés:
Normalisé: عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه
Tokens: ['عاد', 'اليكم', 'من', 'جديد', 'واخيرا', 'درت', 'جيم', 'لهاد', 'الصفحه']
--------------------------------------------------
Normalisé: الله يخلف عليك سيدي محمد
Tokens: ['الله', 'يخلف', 'عليك', 'سيدي', 'محمد']
--------------------------------------------------
Normalisé: هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودليل انعدام غرام ديال المنطق
Tokens: ['هجومك', 'يا', 'اخي', 'علي', 'حسن', 'طارق', 'تبرهيش', 'و', 'قله', 'عقل', 'ودليل', 'انعدام', 'غرام', 'ديال', 'المنطق']
--------------------------------------------------
Normalisé: امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم
Tokens: ['امين', 'عام', 'حزب', 'سياسي', 'كينشر', 'تدوينه', 'وكيدير', 'ليها', 'جيم']
--------------------------------------------------
Normalisé: اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما بقات هضره عليها
Tokens: ['اجي', 'بعدا', 'فين', 'غبرات', 'واحد', 'الحمي', 'القلاعيه', 'شفت', 'ما', 'بقات', 'هضره', 'عليها']
----------

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.10 stopwords.</p>

In [25]:
 #Définir une liste de stopwords en arabe/darija
# Cette liste est un exemple, vous devrez la personnaliser pour la darija marocaine
try:
    arabic_stopwords = set(stopwords.words('arabic'))
except LookupError:
    nltk.download('stopwords')
    arabic_stopwords = set(stopwords.words('arabic'))



In [26]:
arabic_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


In [27]:
def remove_stopwords(tokens):
    if not tokens:
        return []
    
    return [token for token in tokens if token not in arabic_stopwords]

df['tokens_no_stopwords'] = df['tokens'].apply(remove_stopwords)

In [28]:
print("Exemples de tweets après suppression des stopwords:")
for i in range(min(5, len(df))):
    print(f"Tokens avec stopwords: {df['tokens'].iloc[i]}")
    print(f"Tokens sans stopwords: {df['tokens_no_stopwords'].iloc[i]}")
    print("-" * 50)

Exemples de tweets après suppression des stopwords:
Tokens avec stopwords: ['عاد', 'اليكم', 'من', 'جديد', 'واخيرا', 'درت', 'جيم', 'لهاد', 'الصفحه']
Tokens sans stopwords: ['اليكم', 'جديد', 'واخيرا', 'درت', 'لهاد', 'الصفحه']
--------------------------------------------------
Tokens avec stopwords: ['الله', 'يخلف', 'عليك', 'سيدي', 'محمد']
Tokens sans stopwords: ['الله', 'يخلف', 'سيدي', 'محمد']
--------------------------------------------------
Tokens avec stopwords: ['هجومك', 'يا', 'اخي', 'علي', 'حسن', 'طارق', 'تبرهيش', 'و', 'قله', 'عقل', 'ودليل', 'انعدام', 'غرام', 'ديال', 'المنطق']
Tokens sans stopwords: ['هجومك', 'اخي', 'علي', 'حسن', 'طارق', 'تبرهيش', 'قله', 'عقل', 'ودليل', 'انعدام', 'غرام', 'ديال', 'المنطق']
--------------------------------------------------
Tokens avec stopwords: ['امين', 'عام', 'حزب', 'سياسي', 'كينشر', 'تدوينه', 'وكيدير', 'ليها', 'جيم']
Tokens sans stopwords: ['امين', 'عام', 'حزب', 'سياسي', 'كينشر', 'تدوينه', 'وكيدير', 'ليها']
---------------------------------------

<a id="1"></a>  
#### <p style="padding:5px;background-color:#d5eae9;margin:0;color:#c27849;font-family:newtimeroman;font-size:150%;text-align:center;border-radius: 50px 50px;overflow:hidden;font-weight:200">2.11 Encoding the Target Classes
.</p>

In [29]:
#Encoding output labels
encoding = preprocessing.LabelEncoder()

#using fit transform on the data
y = encoding.fit_transform(df.Dialect.values)

#replace dialec column with the encoded
df['Dialect'] = y

In [30]:
df.head(15)

,text,Dialect,text_1,tweets_normalized,tokens,tokens_no_stopwords
0,عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه,1,عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه,عاد اليكم من جديد واخيرا درت جيم لهاد الصفحه,"[عاد, اليكم, من, جديد, واخيرا, درت, جيم, لهاد,...","[اليكم, جديد, واخيرا, درت, لهاد, الصفحه]"
1,الله يخلف عليك سيدي محمد,1,الله يخلف عليك سيدي محمد,الله يخلف عليك سيدي محمد,"[الله, يخلف, عليك, سيدي, محمد]","[الله, يخلف, سيدي, محمد]"
2,هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...,1,هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...,هجومك يا اخي علي حسن طارق تبرهيش و قله عقل ودل...,"[هجومك, يا, اخي, علي, حسن, طارق, تبرهيش, و, قل...","[هجومك, اخي, علي, حسن, طارق, تبرهيش, قله, عقل,..."
3,امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم,1,امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم,امين عام حزب سياسي كينشر تدوينه وكيدير ليها جيم,"[امين, عام, حزب, سياسي, كينشر, تدوينه, وكيدير,...","[امين, عام, حزب, سياسي, كينشر, تدوينه, وكيدير,..."
4,اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...,1,اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...,اجي بعدا فين غبرات واحد الحمي القلاعيه شفت ما ...,"[اجي, بعدا, فين, غبرات, واحد, الحمي, القلاعيه,...","[اجي, فين, غبرات, الحمي, القلاعيه, شفت, بقات, ..."
5,انفلوانزا الخنازير مسؤول عليها وزير الصحه والح...,1,انفلوانزا الخنازير مسؤول عليها وزير الصحه والح...,انفلوانزا الخنازير مسؤول عليها وزير الصحه والح...,"[انفلوانزا, الخنازير, مسؤول, عليها, وزير, الصح...","[انفلوانزا, الخنازير, مسؤول, عليها, وزير, الصح..."
6,انفلوانزا الخنازير تهدد الانسان والحمي القلاعي...,1,انفلوانزا الخنازير تهدد الانسان والحمي القلاعي...,انفلوانزا الخنازير تهدد الانسان والحمي القلاعي...,"[انفلوانزا, الخنازير, تهدد, الانسان, والحمي, ا...","[انفلوانزا, الخنازير, تهدد, الانسان, والحمي, ا..."
7,واحد السؤال وما تديروهاش مني قله الصواب واش بن...,1,واحد السؤال وما تديروهاش مني قله الصواب واش بن...,واحد السؤال وما تديروهاش مني قله الصواب واش بن...,"[واحد, السؤال, وما, تديروهاش, مني, قله, الصواب...","[السؤال, تديروهاش, مني, قله, الصواب, واش, بنكي..."
8,شكرا يوسف سفري بوعميره وبنحليب ديما راجا,1,شكرا يوسف سفري بوعميره وبنحليب ديما راجا,شكرا يوسف سفري بوعميره وبنحليب ديما راجا,"[شكرا, يوسف, سفري, بوعميره, وبنحليب, ديما, راجا]","[شكرا, يوسف, سفري, بوعميره, وبنحليب, ديما, راجا]"
9,الدوله اللي تعاقدات مع رحال باش توكل السجناء و...,1,الدوله اللي تعاقدات مع رحال باش توكل السجناء و...,الدوله اللي تعاقدات مع رحال باش توكل السجناء و...,"[الدوله, اللي, تعاقدات, مع, رحال, باش, توكل, ا...","[الدوله, اللي, تعاقدات, رحال, باش, توكل, السجن..."


In [31]:
data_after_prepro = df[['tokens_no_stopwords', 'Dialect']]


In [34]:
data_after_prepro.to_csv('dataset/dataset_after_preprocessing.csv', index=False, encoding='utf-8-sig')


In [35]:
data_after_prepro

,tokens_no_stopwords,Dialect
0,"[اليكم, جديد, واخيرا, درت, لهاد, الصفحه]",1
1,"[الله, يخلف, سيدي, محمد]",1
2,"[هجومك, اخي, علي, حسن, طارق, تبرهيش, قله, عقل,...",1
3,"[امين, عام, حزب, سياسي, كينشر, تدوينه, وكيدير,...",1
4,"[اجي, فين, غبرات, الحمي, القلاعيه, شفت, بقات, ...",1
...,...,...
24697,"[الله, يدمر]",2
24698,"[سي, امير, راك, ماسط, ماتضحكش, جملا, والله]",2
24699,"[الب, منظرك, وهف, لييك, يكب, سعدك, علي, ماهو, ...",2
24700,"[انسانه, فاشله, نجحت, بالفساد, لتعمل, ربي, يهدي]",2
